In [1]:
import cv2
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

array_of_img = []
file_names = []
gender_labels = []
def read_directory(directory_name):
    for filename_1 in range(1, 41):
        for filename_2 in range(1, 11): 
            img = cv2.imread(directory_name + "/" + str(filename_1) + '_' + str(filename_2) + ".png", 0)
            if filename_1 in [1, 8, 10, 32]:
                gender_labels.append(0)
            else:
                gender_labels.append(1)
            array_of_img.append(img)
    arr = np.array(array_of_img)
    arr = arr.reshape(400, 2576)
    data = pd.DataFrame(arr)
    data['gender'] = [g for g in gender_labels]
    return arr, gender_labels, data

arr, gender_labels, data = read_directory("/Users/nuo/Documents/DA/HW2/ORL Faces")

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(arr, gender_labels, test_size=0.3, random_state=42)

In [4]:
# LR
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(X_train, y_train)

In [5]:
predictions = lr.predict(X_test)
accuracy = lr.score(X_test, y_test)
print("LR accuracy:", accuracy)

LR accuracy: 0.9583333333333334


In [6]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

accuracy_knn = 0
k_highest = 0
prediction = []
for i in range(2, 10):
    knn = KNeighborsClassifier(n_neighbors = i)
    model_knn = knn.fit(X_train, y_train)
    predict_knn = model_knn.predict(X_test)
    acc = accuracy_score(y_test, predict_knn)
    if acc > accuracy_knn:
        accuracy_knn = acc
        k_highest = i
        prediction = predict_knn

In [7]:
print("KNN accuracy:", accuracy_knn, ', k =', k_highest, )

KNN accuracy: 0.9916666666666667 , k = 2


In [8]:
# SVM
from sklearn.svm import SVC

svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [9]:
predict_svm = svm.predict(X_test)
accuracy = accuracy_score(y_test, predict_svm)
print("SVM accuracy:", accuracy)

SVM accuracy: 0.9583333333333334


KNN has the best accuracy in this case.

三者表現：KNN(0.9917) > LR = SVM(0.9583)，推測原因是因為ORL face數據的分佈特性：
1. KNN算法對於密集分布的數據集，可以找到最近的K個鄰居，較容易對數據進行分類(例如上述得出K=2有較好的結果)，所以KNN算法可以取得較好的分類效果。

2. 複雜的決策邊界：KNN算法可以適應複雜的決策邊界，因為它不需要假設數據的分佈特徵，而是通過找到最近的K個鄰居，來決定分類標籤。因此像是ORL face數據的非線性分類問題，KNN算法表現較好。

In [10]:
from sklearn.decomposition import PCA

In [11]:
# LogisticRegression
for i in range(10, 100):
    pca = PCA(n_components = i)
    pca.fit(X_train)
    X_pca = pca.transform(X_train)
    X_pca_test = pca.transform(X_test)
    lr = LogisticRegression().fit(X_pca, y_train)
    predictions = lr.predict(X_pca_test)
    accuracy = lr.score(X_pca_test, y_test)
    if accuracy > 0.95:
        print(accuracy, i)

0.9583333333333334 60
0.9583333333333334 67


根據上述輸出結果：

n_components = [60, 67], accuracy = 0.9583 = the same accuracy of LR in Q1

In [12]:
# KNN
for i in range(10, 100):
    pca = PCA(n_components=i)
    pca.fit(X_train)
    X_pca = pca.transform(X_train)
    X_pca_test = pca.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors = 2)
    model_knn = knn.fit(X_pca, y_train)
    predict_knn = model_knn.predict(X_pca_test)
    acc = accuracy_score(y_test, predict_knn)
    if acc > 0.99:
        print(acc, i)

0.9916666666666667 13
0.9916666666666667 14
0.9916666666666667 15
0.9916666666666667 24
0.9916666666666667 25
0.9916666666666667 26
0.9916666666666667 27
0.9916666666666667 59
0.9916666666666667 63
0.9916666666666667 64
0.9916666666666667 65
0.9916666666666667 70
0.9916666666666667 72
0.9916666666666667 75
0.9916666666666667 76
0.9916666666666667 77
0.9916666666666667 79
0.9916666666666667 84
0.9916666666666667 87
0.9916666666666667 88
0.9916666666666667 91
0.9916666666666667 93
0.9916666666666667 95
0.9916666666666667 96
0.9916666666666667 97
0.9916666666666667 98
0.9916666666666667 99


n_components = [13, 14, 15, 24, 25, 26, 27, 59, 63, 64, 70, 72, 75, 76, 77, 79, 84, 87, 88, 91, 93, 95, 96, 97, 98, 99], 

accuracy = 0.99167 = the same accuracy of KNN in Q1

In [13]:
# SVM
for i in range(1, 12):
    pca = PCA(n_components=i)
    pca.fit(X_train)
    X_pca = pca.transform(X_train)
    X_pca_test = pca.transform(X_test)
    
    svm_pca = SVC(kernel='rbf', probability=True)
    svm_pca.fit(X_pca, y_train)
    predict_svm = svm_pca.predict(X_pca_test)
    acc = accuracy_score(y_test, predict_svm)   
    if acc > 0.95:
        print(acc, i)

0.9583333333333334 11


n_components = 11, accuracy = 0.9853 = the same accuracy of SVM in Q1

In [14]:
mpg = open("/Users/nuo/Documents/DA/HW3/auto-mpg.data.txt")
df = pd.DataFrame(columns = ['car name', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin'])
mpg_list = []
cars = []
for i in mpg:
    a = i.split("\'")
    mpg_list.append(a[0])
    
for data in mpg_list:
    data = data.split("\"")
    cars.append(data)

for c in cars:
    nums = c[0].strip("\t")
    nums = nums.split(" ")
    informations = []
    informations.append(c[1])
    for n in nums:
        if n != " " and n != "" and n != "?":
            try:
                informations.append(int(n))
            except ValueError:
                informations.append(float(n))
        if n == "?":
            informations.append(np.nan)
    df.loc[len(df)] = informations
df.set_index('car name', inplace = True)

# fill the ? number with mean
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].mean())
origin_df = df['origin']
df = df.drop(['origin'], axis = 1)

In [15]:
X_train_q3, X_test_q3, y_train_q3, y_test_q3 = train_test_split(df, origin_df, test_size=0.3, random_state=42)

In [16]:
# LR
lr_q3 = LogisticRegression(multi_class = 'multinomial', solver='lbfgs')
lr_q3.fit(X_train_q3, y_train_q3)
lr_pred = lr_q3.predict(X_test_q3)
lr_acc = accuracy_score(y_test_q3, lr_pred)
print("LR accuracy:", lr_acc)

LR accuracy: 0.7333333333333333


In [17]:
# KNN
accuracy_knn = 0
k_highest = 0
prediction = []
for i in range(2, 10):
    knn = KNeighborsClassifier(n_neighbors = i)
    model_knn = knn.fit(X_train_q3, y_train_q3)
    predict_knn = model_knn.predict(X_test_q3)
    acc = accuracy_score(y_test_q3, predict_knn)
    if acc > accuracy_knn:
        accuracy_knn = acc
        k_highest = i
        prediction = predict_knn

In [18]:
print("KNN accuracy:" , accuracy_knn, ', k =', k_highest)

KNN accuracy: 0.7583333333333333 , k = 5


In [19]:
# SVM
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train_q3, y_train_q3)
predict_svm = svm.predict(X_test_q3)
acc = accuracy_score(y_test_q3, predict_svm)
print("SVM accuracy:" ,acc)

SVM accuracy: 0.7666666666666667


SVM has the best accuracy in this case.

三者表現：SVM(0.7667) > LR = KNN(0.7583)，推測原因是因為auto-mpg數據的分佈特性：
1. 找出最大化類別間邊緣的決策邊界：SVM可以處理具有複雜非線性邊界的數據集，並且比其他分類器對異常值不太敏感。
2. SVM包含線性、多項式、基於半徑等函數，可以將數據轉換成一個更高維度的空間，使得類別更容易分離。所以提高SVM在數據集上的性能。